In [1]:
import open3d as o3d
import numpy as np
import sys


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.append('../sample')
from readdatas import *

In [3]:
path = 'D:\project\Point_Datas\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [4]:
o3d.visualization.draw_geometries([pcd.pcd])

In [4]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [5]:
ground = gpf.np_to_o3d(pcd.ground)

In [6]:
# 计算法向量
ground.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# 计算FPFH描述符
radius_normal = 0.1
radius_feature = 0.25


In [7]:

fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    ground,
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))


In [8]:
feature_vector = fpfh.data.T

In [10]:
reg = ReGrowSegment()
reg.set_data(pcd)
reg._process_data()

[Open3D INFO] Picked point #15962 (-7.1, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #43296 (0.72, -11., -1.5) to add in queue.
[Open3D INFO] Picked point #58037 (4.7, -7.9, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


In [11]:
paves = ground.select_by_index(pcd.paves)
o3d.visualization.draw_geometries([paves])

In [20]:
y = np.full(len(pcd.ground),False,dtype=bool)
y[pcd.paves] = True

In [9]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
# 创建一个SVM模型，使用RBF核函数
clf = svm.SVC(kernel='rbf', C=1, gamma=0.5)

# 训练模型
clf.fit(feature_vector, y)

SVC(C=1, gamma=0.5)

In [11]:

import joblib
# joblib.dump(clf, 'SVM_model_fpfh.pkl')
clf = joblib.load('SVM_model_fpfh.pkl')

In [12]:
# y_pred = clf.predict(feature_vector[pcd.paves])
y_pred_all = clf.predict(feature_vector)

In [23]:
accuracy_all = accuracy_score(y, y_pred_all)

# 输出准确率
print("Accuracy: %0.2f" % accuracy_all)

Accuracy: 0.86


In [26]:
index = np.array(range(len(pcd.ground)))
index = index[y_pred_all]

In [27]:
paves = ground.select_by_index(index)
o3d.visualization.draw_geometries([paves])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
